In [101]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
import numpy as np 
from quantopian.pipeline.factors import CustomFactor,Returns,morningstar
from quantopian.pipeline import Pipeline
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.interactive.data.psychsignal import stocktwits
from quantopian.research import run_pipeline
import pandas as pd
import matplotlib.pyplot as plt
from quantopian.pipeline.data import morningstar
import talib

<h3>Predicting NUGT's price</h3>

In [102]:
price_gld = get_pricing('GLD',start_date='2014-01-25', end_date='2016-01-25')
price_nugt = get_pricing('NUGT',start_date='2014-01-25', end_date='2016-01-25')

In [103]:
price_nugt.drop(['open_price', u'high', u'low',u'volume', u'price'],axis=1,inplace=True)
price_gld.drop(['open_price', u'high', u'low',u'volume', u'price'],axis=1,inplace=True)

In [104]:
price_nugt.columns=['close_price_nugt']
price_spy.columns =['close_price_spy']
price_gld.columns =['close_price_gld']

In [106]:
all_price = pd.concat([price_gld,price_nugt,price_spy],axis =1)

In [107]:
all_price.head()

,close_price_gld,close_price_nugt,close_price_spy
2014-01-27 00:00:00+00:00,120.94,341.0,171.059
2014-01-28 00:00:00+00:00,120.95,361.0,172.078
2014-01-29 00:00:00+00:00,122.46,391.9,170.406
2014-01-30 00:00:00+00:00,119.77,365.4,172.328
2014-01-31 00:00:00+00:00,120.09,364.7,171.155


<h3>Getting % returns of both the following day as well as the previous day for NUGT as well as the gld's previous day return </h3>

In [109]:
all_price['pct_change_gld'] = all_price['close_price_gld'][1:].pct_change()
all_price['pct_change_nugt']= all_price['close_price_nugt'][1:].pct_change()
all_price['pct_change_nugt_shifted']= all_price['close_price_nugt'].shift(-1)[1:].pct_change() 

In [110]:
all_price.head()

,close_price_gld,close_price_nugt,close_price_spy,pct_change_gld,pct_change_nugt,pct_change_nugt_shifted
2014-01-27 00:00:00+00:00,120.94,341.0,171.059,NaN,NaN,NaN
2014-01-28 00:00:00+00:00,120.95,361.0,172.078,NaN,NaN,NaN
2014-01-29 00:00:00+00:00,122.46,391.9,170.406,0.012484,0.085596,-0.067619
2014-01-30 00:00:00+00:00,119.77,365.4,172.328,-0.021966,-0.067619,-0.001916
2014-01-31 00:00:00+00:00,120.09,364.7,171.155,0.002672,-0.001916,-0.018371


In [111]:
all_price.drop(['close_price_gld','close_price_nugt','close_price_spy'],inplace=True,axis=1)

In [112]:
all_price.dropna(inplace=True)

In [113]:
def make_dummies(col):
    return 1 if col > 0 else 0

In [114]:
all_price['pct_change_gld'] = all_price['pct_change_gld'].apply(make_dummies)
all_price['pct_change_nugt']= all_price['pct_change_nugt'].apply(make_dummies)
all_price['pct_change_nugt_shifted'] = all_price['pct_change_nugt_shifted'].apply(make_dummies)

In [115]:
all_price.head()

,pct_change_gld,pct_change_nugt,pct_change_nugt_shifted
2014-01-29 00:00:00+00:00,1,1,0
2014-01-30 00:00:00+00:00,0,0,0
2014-01-31 00:00:00+00:00,1,0,0
2014-02-03 00:00:00+00:00,1,0,1
2014-02-04 00:00:00+00:00,0,1,0


In [116]:
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split

In [117]:
X = all_price[['pct_change_gld','pct_change_nugt']]
y = all_price['pct_change_nugt_shifted']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

In [118]:
lg = LogisticRegression()

In [119]:
lg.fit(X_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0)

In [120]:
predictions = lg.predict(X_test)

<h3>The results are okay but when I backtested the strategy during different years it didn't perform well </h3>

In [122]:
print(classification_report(y_test,predictions))

             precision    recall  f1-score   support

          0       0.61      0.63      0.62        84
          1       0.51      0.48      0.50        66

avg / total       0.56      0.57      0.57       150

